<a href="https://colab.research.google.com/github/tanoManzo/hesso/blob/main/EmoMap_dataset_translation_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Translation Spanish-English

install libraries

In [1]:
! pip install transformers -q
! pip install sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00


load libraries

In [2]:
from transformers import pipeline
pipe_translation = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")
print(pipe_translation(["Hola","es un placer"]))

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Hello.'}, {'translation_text': "It's a pleasure."}]


## Model for emotion detection

possible dataset for fine-tuning

In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/HESSO/EmoMap/primary-plutchik-wheel-DFE.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2524 entries, 0 to 2523
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   _unit_id            2524 non-null   int64  
 1   _golden             2524 non-null   bool   
 2   _unit_state         2524 non-null   object 
 3   _trusted_judgments  2524 non-null   int64  
 4   _last_judgment_at   2500 non-null   object 
 5   emotion             2524 non-null   object 
 6   emotion:confidence  2524 non-null   float64
 7   emotion_gold        54 non-null     object 
 8   id                  2524 non-null   int64  
 9   idiom_id            2524 non-null   int64  
 10  sentence            2524 non-null   object 
dtypes: bool(1), float64(1), int64(4), object(5)
memory usage: 199.8+ KB


load libraries and models

In [4]:
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers.models.realm.retrieval_realm import AutoTokenizer
model_ckpt = "JuliusAlphonso/distilbert-plutchik"

create pipe for emotions detection

In [44]:
pipe_emotion = pipeline("text-classification",model=model_ckpt, top_k=None, max_length=512)
print(pipe_emotion('Hey how is it going? I hope you are doing just fine!'))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'joy', 'score': 0.4039621353149414}, {'label': 'anticipation', 'score': 0.37908849120140076}, {'label': 'trust', 'score': 0.2882962226867676}, {'label': 'anger', 'score': 0.0560523197054863}, {'label': 'surprise', 'score': 0.043713636696338654}, {'label': 'fear', 'score': 0.042806610465049744}, {'label': 'neutral', 'score': 0.021466493606567383}, {'label': 'disgust', 'score': 0.014209728688001633}, {'label': 'sadness', 'score': 0.009838580153882504}]]


full pipe sample

In [45]:
res = pipe_translation("Con cada gesto cariñoso, Laura demostraba su amor hacia su familia.")

In [46]:
res[0]['translation_text']

'With every loving gesture, Laura showed her love for her family.'

In [47]:
pipe_emotion(res[0]['translation_text'])

[[{'label': 'joy', 'score': 0.9037294983863831},
  {'label': 'trust', 'score': 0.30874553322792053},
  {'label': 'anticipation', 'score': 0.05344287306070328},
  {'label': 'surprise', 'score': 0.025506330654025078},
  {'label': 'sadness', 'score': 0.022251101210713387},
  {'label': 'anger', 'score': 0.009547067806124687},
  {'label': 'disgust', 'score': 0.009355872869491577},
  {'label': 'fear', 'score': 0.0074730827473104},
  {'label': 'neutral', 'score': 0.0050794826820492744}]]

In [48]:
sample = [
    "María se puso muy feliz cuando recibió la noticia de que había sido aceptada en la universidad.",
    "Después de ver la película, Juan lloró debido a la emotiva historia.",
    "Carlos se enojó mucho cuando su hermano rompió su juguete favorito.",
    "Con cada gesto cariñoso, Laura demostraba su amor hacia su familia.",
    "Ana sintió miedo cuando escuchó ruidos extraños en la casa durante la noche.",
    "¡Qué sorpresa tan agradable fue ver a mi mejor amiga después de tanto tiempo sin vernos!",
    "La madre mostró preocupación por la salud de su hijo enfermo.",
    "Agradezco enormemente tu ayuda, sin ti no habría podido hacerlo.",
    "Los aficionados celebraron con euforia la victoria de su equipo en el campeonato.",
    " 'Porque eran las tierras que estaban en las orillas del río de la Magdalena y donde habían labranzas, donde habían cultivos de PANCOGER, donde habían cultivos de cacao, o donde habían cultivos donde la comunidad se empeñaba, perfectamente tenía su capacidad productiva, su sostenimiento, su pancoger, tenía su ganadería pero ocurre que esas tierras que eran las más productivas del departamento, se inundaron y al inundarse, pues dejaron una estela de tragedia, dejaron una estela de inconformidad, una estela que hoy ha generado un estigma para el departamento del Huila, un estigma que no hemos podido superar, prueba de ello es que hoy aproximadamente veinte mil personas están siendo afectadas, no solamente desde el punto de vista económico, sino desde el punto de vista psicológico, de la unidad familiar, porque lo que pasó con este proceso, fue un proceso de desplazamiento forzado como lo ha dicho la agencia mundial de represas, que todo proyecto hidroeléctrico en el mundo genera automáticamente un proceso de desplazamiento forzado que fue lo que sucedió aquí, uno escucha historias de personas que la familia eran siete, ocho integrantes, hoy no hay uno con uno, es decir, la familia, la unidad familiar se desintegró en su totalidad, la expectativa del departamento del huila como ente territorial, para efectos de la licencia y uno de los puntos que nosotros estamos tratando de recuperar, por decirlo así, desafortunadamente en una lucha solitaria, porque el gobierno nacional prácticamente nos ha dejado huérfanos, el gobierno nacional en este concepto de estado neoliberal pues prácticamente lo que le interesa es generar energía, no importa a cuantas violaciones de derechos humanos haya, no importa qué suceda con la población, no importa qué suceda con el territorio, lo que le interesa es avasallar, represar para un privado el recurso hídrico y entregarle tal vez el bien, un bien que constitucionalmente es de la nación, que es el agua y entregárselo para que genere capital, ¿genere capital cómo?',"
]

translations = [item['translation_text'] for item in pipe_translation(sample)]
emotions = pipe_emotion(translations)
emotions

[[{'label': 'joy', 'score': 0.9097594618797302},
  {'label': 'trust', 'score': 0.06380993872880936},
  {'label': 'anticipation', 'score': 0.06207909435033798},
  {'label': 'surprise', 'score': 0.05384444445371628},
  {'label': 'neutral', 'score': 0.027514411136507988},
  {'label': 'sadness', 'score': 0.016786852851510048},
  {'label': 'disgust', 'score': 0.00892258808016777},
  {'label': 'anger', 'score': 0.007471565622836351},
  {'label': 'fear', 'score': 0.0068083410151302814}],
 [{'label': 'sadness', 'score': 0.8681671619415283},
  {'label': 'joy', 'score': 0.19633173942565918},
  {'label': 'fear', 'score': 0.051393210887908936},
  {'label': 'disgust', 'score': 0.05115249380469322},
  {'label': 'trust', 'score': 0.0496169775724411},
  {'label': 'anger', 'score': 0.03727302700281143},
  {'label': 'anticipation', 'score': 0.02554301545023918},
  {'label': 'surprise', 'score': 0.023895014077425003},
  {'label': 'neutral', 'score': 0.016677796840667725}],
 [{'label': 'anger', 'score': 0

In [43]:
from nltk.tokenize import word_tokenize

text = 'Porque eran las tierras que estaban en las orillas del río de la Magdalena y donde habían labranzas, donde habían cultivos de PANCOGER, donde habían cultivos de cacao, o donde habían cultivos donde la comunidad se empeñaba, perfectamente tenía su capacidad productiva, su sostenimiento, su pancoger, tenía su ganadería pero ocurre que esas tierras que eran las más productivas del departamento, se inundaron y al inundarse, pues dejaron una estela de tragedia, dejaron una estela de inconformidad, una estela que hoy ha generado un estigma para el departamento del Huila, un estigma que no hemos podido superar, prueba de ello es que hoy aproximadamente veinte mil personas están siendo afectadas, no solamente desde el punto de vista económico, sino desde el punto de vista psicológico, de la unidad familiar, porque lo que pasó con este proceso, fue un proceso de desplazamiento forzado como lo ha dicho la agencia mundial de represas, que todo proyecto hidroeléctrico en el mundo genera automáticamente un proceso de desplazamiento forzado que fue lo que sucedió aquí, uno escucha historias de personas que la familia eran siete, ocho integrantes, hoy no hay uno con uno, es decir, la familia, la unidad familiar se desintegró en su totalidad, la expectativa del departamento del huila como ente territorial, para efectos de la licencia y uno de los puntos que nosotros estamos tratando de recuperar, por decirlo así, desafortunadamente en una lucha solitaria, porque el gobierno nacional prácticamente nos ha dejado huérfanos, el gobierno nacional en este concepto de estado neoliberal pues prácticamente lo que le interesa es generar energía, no importa a cuantas violaciones de derechos humanos haya, no importa qué suceda con la población, no importa qué suceda con el territorio, lo que le interesa es avasallar, represar para un privado el recurso hídrico y entregarle tal vez el bien, un bien que constitucionalmente es de la nación, que es el agua y entregárselo para que genere capital, ¿genere capital cómo?'

len(word_tokenize(text))

364

## Plots

change format for plot

In [ ]:
labels = [item['label'] for item in emotions[0]]
dict_emo = {}

for key in ['sentence']+labels:
  dict_emo[key]=[]

for sent, emos in zip(translations,emotions):
  dict_emo['sentence'].append(sent)

  for emo in emos:
    dict_emo[emo['label']].append(emo['score'])

dict_emo

{'sentence': ['Mary was very happy when she received the news that she had been accepted into college.',
  'After watching the film, John wept because of the emotional story.',
  'Carlos got very angry when his brother broke his favorite toy.',
  'With every loving gesture, Laura showed her love for her family.',
  'Ana was afraid when she heard strange noises in the house during the night.',
  'What a pleasant surprise to see my best friend after so long without seeing us!',
  'The mother showed concern for the health of her sick child.',
  "I'm so grateful for your help, I couldn't have done it without you.",
  'Fans celebrated with euphoria the victory of their team in the championship.',
  'Despite the difficulties, he always kept the hope of a better future.'],
 'joy': [0.9097594618797302,
  0.19633173942565918,
  0.0041679204441607,
  0.9037294983863831,
  0.009992401115596294,
  0.13097751140594482,
  0.014678110368549824,
  0.16805782914161682,
  0.8778924345970154,
  0.2173711

animate plot

In [ ]:
df = pd.DataFrame(dict_emo)
plot_data = pd.melt(df, id_vars='sentence', var_name='attribute', value_name='attribute_value')

In [ ]:
import plotly.express as px

fig = px.line_polar(plot_data,
                    r='attribute_value',
                    theta='attribute',
                    line_close=True,
                    animation_frame = 'sentence',
                    title='Transcription',
                    template = 'ggplot2'
             )
fig.update_layout(
    autosize=False,
    height=600,
    width=1000
)

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000

fig.show()

average plot

In [ ]:
import plotly.graph_objects as go

categories = ['anticipation','joy','trust', 'surprise',
              'fear', 'anger', 'sadness', 'disgust', 'neutral']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[.054, .903, .3087, .205, .007, .007, .02, .009, .005],
      theta=categories,
      fill='toself',
      name='Product A'
))


fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=False
)

fig.show()

## Import transcription

In [10]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha256=7b9d17fcd39f5de2f54426890b9ec2d42c9d201857630707dd62f38054d50193
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [11]:
from docx import Document

file_name = "/content/drive/MyDrive/HESSO/EmoMap/transcripts/TRANSCRIPCIÓN JAVIER ROA.docx"


def getText(filename):
    doc = Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)



In [49]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

transcription = getText(file_name).split('\n')
list_parag_transcriptio = []

for parag in transcription:
  sentences = sent_tokenize(parag)
  list_parag_transcriptio.extend(sentences)

### testing

In [90]:
translations = [item['translation_text'] for item in pipe_translation(list_parag_transcriptio[0:6])]
print(f'translation: {translations}')

emotions = pipe_emotion(translations)
print(f'emotion: {emotions}')

translation: ['TRANSCRIPTION JAVIER ROA INTERVIEW JANUARY', 'T: The first question is that you present yourself and present the place or territory you are going to tell us about.', 'J: Okay', 'T: And what is your relationship with this territory?', 'J: Okay, my name is Javier Roa Salazar, I am a lawyer, environmentalist, I was a councillor of the city of Neiva on two occasions at this time I have been performing for six years as a mining-energy advisor to the secretariat of agriculture of the department of the Huila and within these issues, I have to address the issues of the hydroelectric project of the Quimbo, the context of the image of a macro image of the hydroelectric project of the Quimbo would be many edges, but basically focusing on the interests of the department would be timely loss of the productive capacity of the center of the department, because with the flood of the eight thousand six hundred hectares, the productive activity of the best lands of the department of Huila

In [91]:
from collections import defaultdict


def mean_emotions(list_emotions):
    dict_emo = defaultdict(list)


    for l_emo in list_emotions:
      for item in l_emo:
        if item['score'] > 0.2:
          dict_emo[item['label']].append(item['score'])


    return dict_emo
mean_emotions(emotions)

defaultdict(list,
            {'neutral': [0.6200298070907593, 0.8851895928382874],
             'anticipation': [0.38728559017181396, 0.20442847907543182],
             'anger': [0.2910275161266327,
              0.51987224817276,
              0.3386401832103729,
              0.2473931759595871],
             'sadness': [0.33699947595596313, 0.6668273210525513],
             'fear': [0.28787437081336975, 0.22274036705493927]})

In [97]:
keys = mean_emotions(emotions).keys()

list_val = []
list_keys = []
for k in keys:
  emo_dict = mean_emotions(emotions)
  if k in emo_dict:
    list_val.append(max(mean_emotions(emotions)[k]))
    list_keys.append(k)

In [98]:
import plotly.graph_objects as go

categories = list_keys

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=list_val,
      theta=categories,
      fill='toself',
      name='Product A'
))


fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=False
)

fig.show()

## END

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/HESSO/EmoMap/primary-plutchik-wheel-DFE.csv')
df['emotion'].value_counts()

Neutral         534
Disapproval     323
Optimism        269
Joy             187
Anticipation    175
Anger           169
Fear            124
Sadness         116
Surprise        102
Ambiguous       100
Contempt         83
Submission       79
Love             67
Remorse          44
Aggression       44
Disgust          43
Trust            33
Awe              32
Name: emotion, dtype: int64